# Project 3 Part 1 (Core)

Rodrigo Arguello-Serrano

# Business Problem

>For this project, I produce a MySQL database on Movies from a subset of IMDB's publicly available dataset. I will use this database to analyze what makes a movie successful and provide recommendations to the stakeholder on how to make a successful movie.

# Specifications


> - Exclude any movie with missing values for genre or runtime
> - Include only full-length movies (titleType = "movie").
> - Include only fictional movies (not from documentary genre)
> - Include only movies that were released 2000 - 2021 (include 2000 and 2021)
> - Include only movies that were released in the United States

<a name="1.1"></a>
<h2> 1. 📒Data Dictionary: IMDb Dataset Details </h2>



**title.akas.tsv.gz** - Contains the following information for titles:
titleId (string) - a tconst, an alphanumeric unique identifier of the title<br>
ordering (integer) – a number to uniquely identify rows for a given titleId<br>
title (string) – the localized title<br>
region (string) - the region for this version of the title<br>
language (string) - the language of the title<br>
types (array) - Enumerated set of attributes for this alternative title. One or more of the following: "alternative", "dvd", "festival", "tv", "video", "working", "original", "imdbDisplay".<br>
attributes (array) - Additional terms to describe this alternative title, not enumerated<br>
isOriginalTitle (boolean) – 0: not original title; 1: original title
<br>
<br>
**title.basics.tsv.gz** - Contains the following information for titles:<br>
tconst (string) - alphanumeric unique identifier of the title<br>
titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)<br>
primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release<br>
originalTitle (string) - original title, in the original language<br>
isAdult (boolean) - 0: non-adult title; 1: adult title<br>
startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year<br>
endYear (YYYY) – TV Series end year. ‘\N’ for all other title types<br>
runtimeMinutes – primary runtime of the title, in minutes<br>
genres (string array) – includes up to three genres associated with the title
<br><br>
**title.ratings.tsv.gz** – Contains the IMDb rating and votes information for titles<br>
tconst (string) - alphanumeric unique identifier of the title<br>
averageRating – weighted average of all the individual user ratings<br>
numVotes - number of votes the title has received<br>
[link text](http://url)
This is [My Kaggle Account](https://www.kaggle.com/ahmetekiz)

## 2. Import Modules

In [4]:
#Import Modules
import pandas as pd
import numpy as np

## 3. Upload datasets

In [52]:
# Assigning dataset urls
basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

In [53]:
# Reading data and assigning to dataframe: basics
basics = pd.read_csv(basics_url, sep='\t', low_memory=False)
# Repalcing missing values key \N with numpy nan objects
basics.replace({'\\N':np.nan},inplace=True)

In [54]:
# Reading data and assigning to dataframe: akas
akas = pd.read_csv(akas_url, sep='\t', low_memory=False)
# Repalcing missing values key \N with numpy nan objects
akas.replace({'\\N':np.nan},inplace=True)

In [55]:
# Reading data and assigning to dataframe: 
ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)
# Repalcing missing values key \N with numpy nan objects
ratings.replace({'\\N':np.nan},inplace=True)

## 4. Akas data clean up

In [56]:
# keep only US movies.
akas = akas[akas['region']=='US']

In [57]:
akas.shape

(1418663, 8)

## 5. Basics data clean up

In [43]:
#Eliminate movies that are null for runtimeMinutes and generes
basics=basics.dropna(subset=['runtimeMinutes','genres'])

In [44]:
basics.shape

(2757990, 9)

In [45]:
#keep only titleType==Movie
basics=basics[basics['titleType']=='movie']

In [46]:
basics.shape

(378185, 9)

In [47]:
basics['startYear'] = basics['startYear'].astype(float)

In [48]:
#keep startYear 2000-2022
basics = basics[(basics['startYear']<=2022.0)&(basics['startYear']>=2000.0)]

In [49]:
basics.shape

(221923, 9)

In [50]:
# Eliminate movies that include "Documentary" in genre (see tip below)
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics= basics[~is_documentary]

In [51]:
basics['tconst'].isin(akas['titleId']).value_counts()

False    146714
Name: tconst, dtype: int64

In [25]:
#Keep only US movies
keepers =basics['tconst'].isin(akas['titleId'])
basics = basics[keepers]

## 5. Ratings data clean up

In [26]:
#Keep only US movies
keepers_r =ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers_r]

In [27]:
# Dropping all rows with nan values from akka
akas = akas.dropna()

## 7. Checking Dataframe Info

In [28]:
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85732 entries, 34803 to 9658557
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          85732 non-null  object 
 1   titleType       85732 non-null  object 
 2   primaryTitle    85732 non-null  object 
 3   originalTitle   85732 non-null  object 
 4   isAdult         85732 non-null  object 
 5   startYear       85732 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  85732 non-null  object 
 8   genres          85732 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.5+ MB


In [35]:
# Dropping all rows with nan values from akka
akas = akas.dropna()

In [30]:
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 102333 to 1051464
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleId          22 non-null     object
 1   ordering         22 non-null     int64 
 2   title            22 non-null     object
 3   region           22 non-null     object
 4   language         22 non-null     object
 5   types            22 non-null     object
 6   attributes       22 non-null     object
 7   isOriginalTitle  22 non-null     object
dtypes: int64(1), object(7)
memory usage: 1.5+ KB


In [31]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 490817 entries, 0 to 1284885
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   tconst         490817 non-null  object 
 1   averageRating  490817 non-null  float64
 2   numVotes       490817 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 15.0+ MB


## 8. Saving Dataframes

### 8.1 Basics Dataframe

In [32]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


### 8.2 Akas Dataframe

In [33]:
## Save current dataframe to file.
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
akas = pd.read_csv("Data/title_akas.csv.gz",compression='gzip')
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0022542,1,Di shtime fun Yisroel,US,yi,alternative,YIVO translation,0
1,tt0024265,4,Geleb un gelakht,US,yi,alternative,modern translation,0
2,tt0024751,9,Avram Ovenu,US,yi,alternative,YIVO translation,0
3,tt0026010,3,Der yidishe Kenigen Lir,US,yi,alternative,YIVO translation,0
4,tt0027911,1,Libe un Laydnshaft,US,yi,alternative,modern translation,0


### 8.3 Ratings Dataframe

In [34]:
## Save current dataframe to file.
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

# Open saved file and preview again
ratings = pd.read_csv("Data/title_ratings.csv.gz",compression='gzip')
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1956
1,tt0000002,5.8,263
2,tt0000005,6.2,2592
3,tt0000006,5.1,177
4,tt0000007,5.4,812
